In [1]:
from test import *
from NNfxns import neural_network
#from data import rap1-lieb-positives
from NNfxns import train
import sys
import matplotlib.pyplot as plt
import sklearn.metrics as skm

In [2]:
"""
Training the neural net

What can be used:
     autoencoder
     %%%%%%%%werk <partitions> <sampling>
     test
Arguments:
    autoencoder
        Run ze autoencoder
    %%%%%%%%werk
        Do the rap1 learning task including cross-validation
    test
        Classify test data and output to tsv file type
    <partitions>
        Number of partitions to make for cross-valitation (k-fold!)
    <sampling>
        Sampling method for NN training input
        (slide) Go over each sequence in 17 nucleotide sliding frame (the length of the positives/test binding sites)
        (space) Each sequence is cut into 17 nucleotide bits for inputs (the binary bits that are useable by our model)
"""

def werk():
    """
    Train neural network on RAP1 binding sites
        * Input layer with 17*4 nodes (because 17 different nucleotides defining each sequence, and 4 different
          possible nucleotides describing those positions) + bias
        * Hidden layer with 23-35 nodes (merge at least 2 input neurons)+ bias
        * One output layer node (number of neurons in the output layer will equal the number of outputs
          associated with each input; we want one answer)
    Train against negative and positive binding sites
        * Import all negative sequences from .fa file
        * For each sequence, iterate every 17 bases and train with
          expected of 0 (since these are negatives, while positives should be 1)
        * Because it is so important to have a 1:1 ratio of negatives:positives when training our model, 
          for every 137 negative training instances, need to train it against all positive binding sites
          with expected of 1 
        * Will continue until negative binding sites have been fully ran through
    """

    # This part takes care of bringing in all those positive sequences
    
    positive_sites = [pos_seq.strip() for pos_seq in open('data/rap1-lieb-positives.txt')]

    
    
    # This part takes care of bringing in all those neg sequences
    
    negative_sites = list(SeqIO.parse('data/yeast-upstream-1k-negative.fa', 'fasta'))
    
    

    # Separate into random sections for the k-fold x-validation for both our positives and negatives 
    # Taken from : http://stackoverflow.com/questions/3352737/python-randomly-partition-a-list-into-n-nearly-equal-parts
    
    
    partitions = int(args['<partitions>'])
    neg_division = len(negative_sites) / float(partitions) # how many partitions we can make from the sites given
    neg_randomly_partitioned_list = [negative_sites[int(round(neg_division * i)): int(round(neg_division * (i + 1)))]
                                     for i in range(partitions)] # makes a list of those partitions thru each site

    pos_division = len(positive_sites) / float(partitions) # ditto ^^
    pos_randomly_partitioned_list = [positive_sites[int(round(pos_division * i)): int(round(pos_division * (i + 1)))]
                                     for i in range(partitions)]

    
    
    # Go thru neg sites subsets for x-validation, keep track of how many separations we do based on partitions
    
    separation = 0
    for index in range(int(args['<partitions>'])):
        # Set up cross-validation sets for the positives and negatives
        neg_site_list_copy = copy.deepcopy(neg_randomly_partitioned_list)
        del neg_site_list_copy[index]
        neg_site_training = [seq for partition in neg_site_list_copy for seq in partition]
        neg_cross_validation_set = neg_randomly_partitioned_list[index]

        pos_site_list_copy = copy.deepcopy(pos_randomly_partitioned_list)
        del pos_site_list_copy[index]
        pos_site_training = [seq for partition in pos_site_list_copy for seq in partition]
        pos_cross_validation_set = pos_randomly_partitioned_list[index]

        print("Training on the training set...")

        # Input our hyperparameters = # nodes
        NN = neural_network(68, 23, 1)

        # See neural_net.py to get info on initialization
        NN.initialize_values()

        pos_counter = 0
        counter = 0

        # If we're sampling from tneg then we'll slide over 17 nucleotides 
        
        if args['<sampling>'] == 'slide':
            for site in neg_site_training:

                # Iterate over site in 17 nucleotide sliding frames in negative sites, decide which model to use
                for block in range(len(site) - 16):
                    slice = site[block:(block + 17)].seq
                    if slice not in positive_sites:
                        if all([slice[4] == 'C', slice[5] == 'C', slice[9] == 'C']) == False:
                            NN.set_input_and_expected_values(slice, autoencoder=False, negative=True)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()
                            pos_counter += 1
                        else:
                            print(slice)

                    if pos_counter == len(pos_site_training):
                        for pos_site in pos_site_training:
                            NN.set_input_and_expected_values(pos_site, autoencoder=False, negative=False)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()

                        pos_counter = 0

                # have reset the positives counter and will now say that we've done some training on those
                
                counter += 1

                print("Training set: {}/{} completed...".format(counter, len(neg_cross_validation_set)))

                max_change_1 = NN.matrix_1_errors.max()
                min_change_1 = NN.matrix_1_errors.min()
                max_change_2 = NN.matrix_2_errors.max()
                min_change_2 = NN.matrix_2_errors.min()

                if any([max_change_1 < 0.00000000001 and max_change_1 > 0,
                        min_change_1 > -.00000000001 and min_change_1 < 0]) and any(
                    [max_change_2 < 0.00000000001 and max_change_2 > 0,
                     min_change_2 > -0.00000000001 and min_change_2 < 0]):
                    print("Stop criterion met after {} iterations".format(counter))
                    break

        #when we sample from the negatives we only take 17 nucleotide chunks from each site
        
        if args['<sampling>'] == 'space':
            for site in neg_site_training:
                
                number_of_blocks = int(len(site) / 17) #length of neg site tells us the amount of 17 length chunks possible

                for block in range(number_of_blocks):
                    slice = site[(block * 17):((block + 1) * 17)].seq
                    if slice not in positive_sites:
                        if all([slice[4] == 'C', slice[5] == 'C', slice[9] == 'C']) == False:
                            NN.set_input_and_expected_values(slice, autoencoder=False, negative=True)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()
                            pos_counter += 1

                        else:
                            print(slice)

                    #quick check to make sure that we've finished going thru the positives yet
                    if pos_counter == len(pos_site_training):
                        for pos_site in pos_site_training:
                            NN.set_input_and_expected_values(pos_site, autoencoder=False, negative=False)
                            NN.forward_propogation()
                            NN.backward_propogation()
                            NN.update_weights_and_bias()

                        pos_counter = 0

                    counter += 1

                max_change_1 = NN.matrix_1_errors.max()
                min_change_1 = NN.matrix_1_errors.min()
                max_change_2 = NN.matrix_2_errors.max()
                min_change_2 = NN.matrix_2_errors.min()

                if any([max_change_1 < 0.00000000001 and max_change_1 > 0,
                        min_change_1 > -.00000000001 and min_change_1 < 0]) and any(
                    [max_change_2 < 0.00000000001 and max_change_2 > 0,
                     min_change_2 > -0.00000000001 and min_change_2 < 0]):
                    print("Stop criterion met after {} iterations".format(counter))
                    break

        # taken each partition and trained model
        print("Performing Cross-validation")

        pos_list = []
        neg_list = []

        
        # Return the sets of positives and negatives from the x-validation
        
        print("Negative cross-validation set...")
        counter = 0
        for site in neg_cross_validation_set:
            for slice in range(len(site) - 16):
                NN.set_input_and_expected_values(site[slice:slice + 17].seq, autoencoder=False, negative=True)
                NN.forward_propogation()
                neg_list.append(NN.output_layer_output)
            counter += 1
            print("Negative cross-validation: {}/{} completed...".format(counter, len(neg_cross_validation_set)))
            break

        print("Positive cross-validation set...")
        for site in pos_cross_validation_set:
            NN.set_input_and_expected_values(site, autoencoder=False)
            NN.forward_propogation()
            pos_list.append(NN.output_layer_output)

        print('Positive avg: {}'.format(sum(pos_list) / len(pos_list)))
        print('Negative avg: {}'.format(sum(neg_list) / len(neg_list)))
        print(NN.matrix_1_bias)
        print(NN.matrix_2_bias)

        # Output the connection matrices with greatest separation between the average positive and negative scores
        if ((sum(pos_list) / len(pos_list)) - (sum(neg_list) / len(neg_list))) > separation:
            np.savetxt('connection_matrix_1.csv', NN.matrix_1_bias, delimiter=',')
            np.savetxt('connection_matrix_2.csv', NN.matrix_2_bias, delimiter=',')
            separation = (sum(pos_list) / len(pos_list)) - (sum(neg_list) / len(neg_list))


# A simple definition of the autoencoder that uses those same NN parameters
def autoencoder():
  
    NN = neural_network()
    NN.set_input_and_expected_values('GA', autoencoder=True)
    NN.initialize_values()

    # Stop criterion
    finished_working = False

    while finished_working == False:
        NN.forward_propogation()
        NN.backward_propogation()
        NN.update_weights_and_bias()

        max_change_1 = NN.matrix_1_errors.max()
        min_change_1 = NN.matrix_1_errors.min()
        max_change_2 = NN.matrix_2_errors.max()
        min_change_2 = NN.matrix_2_errors.min()

        if any([max_change_1 < 0.00001 and max_change_1 > 0,
                min_change_1 > -.00001 and min_change_1 < 0]) or any(
            [max_change_2 < 0.00001 and max_change_2 > 0,
             min_change_2 > -0.00001 and min_change_2 < 0]):
            finished_working = True

    print(NN.output_layer_output)

def test():
    test_sequences = open('data/eightBit.txt')
    NN = auto()
    NN.matrix_1_bias = np.loadtxt('connection_matrix_1.csv', delimiter=',')
    NN.matrix_2_bias = np.loadtxt('connection_matrix_2.csv', delimiter=',')

    NN_outputs = open('NN_predictions.txt', 'w')

    for test_seq in test_sequences:
        NN.set_input_and_expected_values(test_seq.strip())
        NN.forward_propogation()
        NN_outputs.write('{}\t{}\n'.format(test_seq.strip(), NN.output_layer_output[0]))

    NN_outputs.close()

if __name__ == 'train':
    import docopt
    import numpy as np
    from Bio import SeqIO
    import copy
    from .NNfxns import neural_network

    args = docopt.docopt(__doc__)

    if args['autoencoder']:
        autoencoder()

    if args['werk']:
        werk()

    if args['test']:
        test()



In [4]:
import numpy as np

"""
-Make a NN class
-define my functions
  -autoencoder and its parameters
  -initial values/input
  -vecotorization
  -expected values
  -output
  -backwards propagation
  -feedforward propagation
  -weights/bias
  -sigmoid function
  -sigmoid derivative

"""
# Make a class to allow self reference
class auto():

# The NN will have a number of input neurons, a hidden layer, and number of an output neurons of the specified numbers, 
# which we apply to the class and can adjust as necessary later

    def __init__(self, input_layer_nodes=8, hidden_layer_nodes=3, output_layer_nodes=8):
        self.input_layer_nodes = input_layer_nodes
        self.hidden_layer_nodes = hidden_layer_nodes
        self.output_layer_nodes = output_layer_nodes

        # We want the autoencoder to learn the best weights for the data so we 
        # initialize matrices with random weights 0.1 > x > -0.1
        # These will link the autoencoder's different layers.
        
        self.connection_matrix_1 = np.random.randn(self.input_layer_nodes, self.hidden_layer_nodes) / 10
        self.connection_matrix_2 = np.random.randn(self.hidden_layer_nodes, self.output_layer_nodes) / 10

        # We have several vectors that our encoder will make to take in (input) and put out (output) data.
        # Recall the hidden layer will reduce the dimension, in a vector-wise fashion
        
        self.input_vector = None
        self.hidden_layer_output = None
        self.output_layer_output = None

        # We can also start off our autoencoder with a biased vector or matrices, depending on what we need from our model
        
        self.input_with_bias = None
        self.matrix_1_bias = None
        self.matrix_2_bias = None

        # Recall that we need to take the derivative of the cost function with respect to weight, and develop matrices from this
        
        self.hidden_dx_matrix = np.zeros([self.hidden_layer_nodes, self.hidden_layer_nodes])
        self.output_dx_matrix = np.zeros([self.output_layer_nodes, self.output_layer_nodes])

        # Hyperparameter: Learning Rate (controls how quickly a NN learns a problem)
        # Typically in a range between [0.1, 1], configureable
        # A perfect learning rate will make the model learn to best approximate the function given available resources (layers/nodes)
        # in a given number of epochs
        # We'll start with a large one because we've got other stuff to do, dudes. The pitfall of that is that it will likely arrive at a suboptimal
        # set of weights.
        
        self.learning_rate = 1

        
        
        # Bit conversion for DNA into NN inputs because I'm too lazy to think of a different way to encode these. Recall, we want either 0 or 1, not anything continuous.
        
        self.base_binary_conversion = {'0': '0',
                                       '1': '1'
                                       }

        
        
        
        # These are the expected values for our NN to return
        
        self.expected_values = None



    # Now we make moves to fill in and interpret those matrices
    
    # Initialize the values of the bias matrix given the information passing through the hidden layer and output layer
    
    def initialize_values(self):
        bias_ones_1 = np.ones([1, self.hidden_layer_nodes])
        self.matrix_1_bias = np.append(self.connection_matrix_1, bias_ones_1, axis=0)

        bias_ones_2 = np.ones([1, self.output_layer_nodes])
        self.matrix_2_bias = np.append(self.connection_matrix_2, bias_ones_2, axis=0)

    # We'll make the binding sites into binary bits that the encoder can interpret, and then tell it what to expect 
    
    def set_input_and_expected_values(self, input_DNA, autoencoder=True, negative=True):
        # Convert input DNA sequence into binary bits
        self._construct_input_vector(input_DNA)
        # Set expected value depending on autoencoder or werk
        self._set_expected_values(autoencoder, negative)
        # Weight matrices and input/output vectors with bias applied
        self.input_with_bias = np.append(self.input_vector, [1])

    
    
    # This directly handles conversion of the DNA binding site string to the 1/0 vector describing it, and assigns it to the original input class
    
    def _construct_input_vector(self, input_DNA):
       
        temp_vector_list = []

        for base in input_DNA:
            for number in self.base_binary_conversion[base]:
                temp_vector_list.append(float(number))

        self.input_vector = np.asarray(temp_vector_list)

    
    
    # This will set the values we expect from the NN depending on whether or not we use the autoencoder (T/F).
    
    def _set_expected_values(self, autoencoder=True, negative=True):
        if autoencoder == True:
            self.expected_values = self.input_vector

        if autoencoder == False:
            if negative == True:
                self.expected_values = 0
            if negative == False:
                self.expected_values = 1

    # Recall an autoencoder is a feedforward method. We need to convert input to hidden layer reduced dim info to output layer
    # that is the same as the input.
    
    def forward_propogation(self):
        # Generates hidden layer outputs
        
        output_one_list = []

        for element in np.nditer(np.dot(self.input_with_bias, self.matrix_1_bias)):
            output_one_list.append(self._sigmoid_function(element))
        self.hidden_layer_output = np.asarray(output_one_list)

        # Calculate the square derivate matrix for the hidden layer outputs
        
        for position, value in enumerate(self.hidden_layer_output):
            self.hidden_dx_matrix[position][position] = self._sigmoid_function_derivative(value)

        # The results from the output layer 
        # Add bias to hidden_layer_output
        
        self.hidden_output_bias = np.append(self.hidden_layer_output, [1])

        output_two_list = []
        for element in np.nditer(np.dot(self.hidden_output_bias, self.matrix_2_bias)):
            output_two_list.append(self._sigmoid_function(element))
        self.output_layer_output = np.asarray(output_two_list)

        # Calculate square derivate matrix for output layer outputs
        
        for position, value in enumerate(self.output_layer_output):
            self.output_dx_matrix[position][position] = self._sigmoid_function_derivative(value)

    
    # Recall that in stochastic gradient descent, we estimate the error gradient for the current state of the model using
    # examples from the training. The backwards propagation is what we use to update the weights of the model. 
    
    def backward_propogation(self):
        # Output Layer error
        
        deviations = self.output_layer_output - self.expected_values

        output_layer_errors = np.dot(self.output_dx_matrix, deviations)

        # Hidden Layer error
        
        hidden_layer_errors = np.dot(np.dot(self.hidden_dx_matrix, self.connection_matrix_2), output_layer_errors)

        # Matrix 2 Errors (those of our output layer)
        
        output_rated_row_vector = np.asmatrix(-(self.learning_rate * output_layer_errors)).transpose()
        matrix_2_errors_transposed = np.dot(output_rated_row_vector, np.asmatrix(self.hidden_output_bias))
        self.matrix_2_errors = matrix_2_errors_transposed.transpose()

        # Matrix 1 Errors (those of our hidden layer)
        
        hidden_rated_row_vector = np.asmatrix(-(self.learning_rate * hidden_layer_errors)).transpose()
        matrix_1_errors_transposed = np.dot(hidden_rated_row_vector, np.asmatrix(self.input_with_bias))
        self.matrix_1_errors = matrix_1_errors_transposed.transpose()

    
    # Basically the sum between the output and hidden layers' bias and errors is what we use...
    
    def update_weights_and_bias(self):
        self.matrix_1_bias = self.matrix_1_bias + self.matrix_1_errors
        self.matrix_2_bias = self.matrix_2_bias + self.matrix_2_errors

        self.connection_matrix_1 = self.matrix_1_bias[:-1]
        self.connection_matrix_2 = self.matrix_2_bias[:-1]

    
    # The activation function for the layers. Some people use relu. Its more of less simple to take the deriv of sigmoid. 
    
    def _sigmoid_function(self, input):
        return float(1 / (1 + np.exp(-input)))

    
    # Recall we also take the derivative of the activatio function. 
    def _sigmoid_function_derivative(self, input):
        return float(self._sigmoid_function(input) * (1 - self._sigmoid_function(input)))



In [11]:
def test_auto():
  
    eightBit = open('data/eightBit.txt')
    NN = auto()
    for bits in eightBit:
        NN.set_input_and_expected_values(bits.strip(), autoencoder=True)
        NN.initialize_values()
        
        # Stop criterion
        finished_working = False

        while finished_working == False:
            NN.forward_propogation()
            NN.backward_propogation()
            NN.update_weights_and_bias()

            max_change_1 = NN.matrix_1_errors.max()
            min_change_1 = NN.matrix_1_errors.min()
            max_change_2 = NN.matrix_2_errors.max()
            min_change_2 = NN.matrix_2_errors.min()

            if any([max_change_1 < 0.00001 and max_change_1 > 0,
                    min_change_1 > -.00001 and min_change_1 < 0]) or any(
                [max_change_2 < 0.00001 and max_change_2 > 0,
                 min_change_2 > -0.00001 and min_change_2 < 0]):
                finished_working = True

        print(NN.output_layer_output)   
    

In [14]:
print(open('data/eightBit.txt'))

<_io.TextIOWrapper name='data/eightBit.txt' mode='r' encoding='UTF-8'>


In [12]:
test_auto()

[9.99949126e-01 3.99995008e-05 3.99955293e-05 3.99962867e-05
 3.99984625e-05 3.99967960e-05 3.99960781e-05 3.99999973e-05]
[3.99985675e-05 9.99946848e-01 2.57926418e-05 2.60418918e-05
 2.59730002e-05 2.59787067e-05 2.59901803e-05 2.61870126e-05]
[3.01333250e-05 3.99988046e-05 9.99939176e-01 1.49154779e-05
 1.51149238e-05 1.49429129e-05 1.48778409e-05 1.52154691e-05]
[1.93195523e-05 2.48948966e-05 3.99988031e-05 9.99929688e-01
 1.03933983e-05 1.03922915e-05 1.03475381e-05 1.02865919e-05]
[0.00193798 0.00299085 0.00481673 0.00698593 0.9829648  0.00125429
 0.0012227  0.00112854]
[9.27584501e-06 1.31990155e-05 1.80325124e-05 3.99995084e-05
 1.97193982e-05 9.99924754e-01 5.72898531e-06 5.85558823e-06]
[8.00878183e-06 1.04457314e-05 1.24600157e-05 1.38026387e-05
 1.72800042e-05 3.99992599e-05 9.99930160e-01 6.26102287e-06]
[4.22246876e-06 5.26472532e-06 6.90258785e-06 8.73540765e-06
 9.41536990e-06 1.97718351e-05 3.99987067e-05 9.99918081e-01]


In [4]:
#use this when determining how well the nn gets scores for the test set
def test():
    test_sequences = open('data/rap1-lieb-test.txt')
    NN = neural_network(68, 23, 1)
    NN.matrix_1_bias = np.loadtxt('connection_matrix_1.csv', delimiter=',')
    NN.matrix_2_bias = np.loadtxt('connection_matrix_2.csv', delimiter=',')

    NN_outputs = open('NN_predictions.txt', 'w')

    for test_seq in test_sequences:
        NN.set_input_and_expected_values(test_seq.strip())
        NN.forward_propogation()
        NN_outputs.write('{}\t{}\n'.format(test_seq.strip(), NN.output_layer_output[0]))

    NN_outputs.close()
